In [23]:
!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

--2024-09-27 03:32:16--  https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.github.com (gist.github.com)... 20.205.243.166
Connecting to gist.github.com (gist.github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py [following]
--2024-09-27 03:32:16--  https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2730 (2.7K) [text/plain]
Saving to: ‘cpp_plugin.py’

cpp_plugin.py       100%[===================>]   2.67K  --.-KB/s    in 0s      

2024-09-27 03:32:1

In [24]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-bch1onrp
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-bch1onrp
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [25]:
%%cuda_group_save -g "source" -n "data_types.h"
# /**
#  * A collection of commonly used data types throughout this project.
#  */
#pragma once

#include <stdint.h> // uint32_t

using element_t = uint32_t;

In [26]:
%%cuda_group_save -g "source" -n "cuda_common.h"
/**
 * Standard macros that can be useful for error checking.
 * https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
 */
#pragma once

#include <cuda.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

In [27]:
%%cuda_group_save -g "source" -n "data_generator.h"
/**
 * Functions for generating random input data with a fixed seed
 */
#pragma once

#include <random>  // for std::mt19937, std::uniform_int_distribution
#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {

/**
 * Generates and returns a vector of random uniform data of a given length, n,
 * for any integral type. Input range will be [0, 2n].
 */
template < typename T >
std::vector< T > generate_uniform( std::size_t n )
{
    // for details of random number generation, see:
    // https://en.cppreference.com/w/cpp/numeric/random/uniform_int_distribution
    std::size_t random_seed = 20240916;  // use magic seed
    std::mt19937 rng( random_seed );     // use mersenne twister generator
    std::uniform_int_distribution<> distrib(0, 2 * n);

    std::vector< T > random_data( n ); // init array
    std::generate( std::begin( random_data )
                 , std::end  ( random_data )
                 , [ &rng, &distrib ](){ return static_cast< T >( distrib( rng ) ); });

    return random_data;
}

} // namespace a1
} // namespace csc485b

In [28]:
%%cuda_group_save -g "source" -n "algorithm_choices.h"
#pragma once

#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {
namespace cpu {

void run_cpu_baseline( std::vector< element_t > data, std::size_t switch_at, std::size_t n );

} // namespace cpu


namespace gpu {

void run_gpu_soln( std::vector< element_t > data, std::size_t switch_at, std::size_t n );

} // namespace gpu
} // namespace a1
} // namespace csc485b

In [29]:
%%cuda_group_save -g "source" -n "cpu_baseline.cu"
/**
 * CPU methods that the GPU should outperform.
 */

#include "algorithm_choices.h"

#include <algorithm> // std::sort()
#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl

namespace csc485b {
namespace a1      {
namespace cpu     {

/**
 * Simple solution that just sorts the whole array with a built-in sort
 * function and then resorts the last portion in the opposing order with
 * a second call to that same built-in sort function.
 */
void opposing_sort( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    std::sort( data, data + num_elements, std::less< element_t >{} );
    std::sort( data + invert_at_pos, data + num_elements, std::greater< element_t >{} );
}

/**
 * Run the single-threaded CPU baseline that students are supposed to outperform
 * in order to obtain higher grades on this assignment. Times the execution and
 * prints to the standard output (e.g., the screen) that "wall time." Note that
 * the functions takes the input by value so as to not perturb the original data
 * in place.
 */
void run_cpu_baseline( std::vector< element_t > data, std::size_t switch_at, std::size_t n )
{
    auto const cpu_start = std::chrono::high_resolution_clock::now();
    opposing_sort( data.data(), switch_at, n );
    auto const cpu_end = std::chrono::high_resolution_clock::now();

    std::cout << "CPU Baseline time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(cpu_end - cpu_start).count()
              << " ns" << std::endl;

    for( auto const x : data ) std::cout << x << " "; std::cout << std::endl;
}

} // namespace cpu
} // namespace a1
} // namespace csc485b

In [30]:
%%cuda_group_save -g "source" -n "gpu_solution.cu"
/**
 * The file in which you will implement your GPU solutions!
 */

#include "algorithm_choices.h"
#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl
#include "cuda_common.h"

namespace csc485b {
namespace a1 {
namespace gpu {


/**
 * The CPU baseline benefits from warm caches because the data was generated on
 * the CPU. Run the data through the GPU once with some arbitrary logic to
 * ensure that the GPU cache is warm too and the comparison is more fair.
 */
__global__
void warm_the_gpu(element_t *data, std::size_t invert_at_pos, std::size_t num_elements) {
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    if (th_id < num_elements && data[th_id] > num_elements * 100) {
        ++data[th_id];
    }
}

/**
 * Bitonic Sort for small arrays (n < 1024)
 */
__device__
void full_sort(element_t *data, std::size_t num_elements,std::size_t invert_at_pos ) {
    // Use shared memory to sort small arrays entirely within a block
    __shared__ element_t s_data[1024];

    int idx = threadIdx.x;

    // Load data into shared memory
    if (idx < num_elements) {
        s_data[idx] = data[idx];
    } else {
        s_data[idx] = INT_MAX;  // Assuming element_t is int
    }
    __syncthreads();

    // Bitonic sort in shared memory
    for (unsigned int k = 2; k <= num_elements; k <<= 1) {
        for (unsigned int j = k >> 1; j > 0; j >>= 1) {
            unsigned int ixj = idx ^ j;

            if (ixj < num_elements) {
                if (ixj > idx) {
                    if ((idx & k) == 0) {
                        if (s_data[idx] > s_data[ixj]) {
                            // Swap
                            element_t temp = s_data[idx];
                            s_data[idx] = s_data[ixj];
                            s_data[ixj] = temp;
                        }
                    } else {
                        if (s_data[idx] < s_data[ixj]) {
                            // Swap
                            element_t temp = s_data[idx];
                            s_data[idx] = s_data[ixj];
                            s_data[ixj] = temp;
                        }
                    }
                }
            }
            __syncthreads();
        }
    }

    // Write sorted data back to global memory
    // Use the calculations to determine correct position
    if (idx >= invert_at_pos && idx < num_elements) {
        data[num_elements - idx + invert_at_pos - 1] = s_data[idx];
    } else if (idx < invert_at_pos && idx < num_elements) {
        data[idx] = s_data[idx];
    }
}

/**
 * Bitonic Sort for larger arrays (n >= 1024) using shared memory
 */
__device__
void shared_full_sort(element_t *data, std::size_t num_elements,std::size_t invert_at_pos ) {
    // Each block sorts its own subarray
    __shared__ element_t s_data[1024];

    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int local_idx = threadIdx.x;

    // Load data into shared memory
    if (idx < num_elements) {
        s_data[local_idx] = data[idx];
    } else {
        s_data[local_idx] = INT_MAX;  // Assuming element_t is int
    }
    __syncthreads();

    // Perform bitonic sort in shared memory
    // blockDim.x should be a power of 2
    for (unsigned int k = 2; k <= blockDim.x; k <<= 1) {
        for (unsigned int j = k >> 1; j > 0; j >>= 1) {
            unsigned int ixj = local_idx ^ j;

            if (ixj > local_idx) {
                if ((local_idx & k) == 0) {
                    if (s_data[local_idx] > s_data[ixj]) {
                        // Swap
                        element_t temp = s_data[local_idx];
                        s_data[local_idx] = s_data[ixj];
                        s_data[ixj] = temp;
                    }
                } else {
                    if (s_data[local_idx] < s_data[ixj]) {
                        // Swap
                        element_t temp = s_data[local_idx];
                        s_data[local_idx] = s_data[ixj];
                        s_data[ixj] = temp;
                    }
                }
            }
            __syncthreads();
        }
    }

    // Write sorted data back to global memory
    // Use the calculations to determine correct position
    if (idx >= invert_at_pos && idx < num_elements) {
        data[num_elements - idx + invert_at_pos - 1] = s_data[local_idx];
    } else if (idx < invert_at_pos && idx < num_elements) {
        data[idx] = s_data[local_idx];
    }
}

/**
 * Main kernel function
 */
__global__
void opposing_sort(element_t *data, std::size_t invert_at_pos, std::size_t num_elements) {
    if (num_elements <= 1024) {
        full_sort(data, num_elements, invert_at_pos);
    } else {
        shared_full_sort(data, num_elements, invert_at_pos);
    }
    __syncthreads();
}

/**
 * Performs all the logic of allocating device vectors and copying host/input
 * vectors to the device. Times the opposing_sort() kernel with wall time,
 * but excludes set up and tear down costs such as mallocs, frees, and memcpies.
 */
void run_gpu_soln(std::vector<element_t> data, std::size_t switch_at, std::size_t n) {
    std::size_t const threads_per_block = 1024;
    std::size_t const num_blocks = (n + threads_per_block - 1) / threads_per_block;

    // Allocate arrays on the device/GPU
    element_t *d_data;
    cudaMalloc((void**)&d_data, sizeof(element_t) * n);
    CHECK_ERROR("Allocating input array on device");

    // Copy the input from the host to the device/GPU
    cudaMemcpy(d_data, data.data(), sizeof(element_t) * n, cudaMemcpyHostToDevice);
    CHECK_ERROR("Copying input array to device");

    // Warm the cache on the GPU for a more fair comparison
    warm_the_gpu<<<num_blocks, threads_per_block>>>(d_data, switch_at, n);

    // Time the execution of the kernel that you implemented
    auto const kernel_start = std::chrono::high_resolution_clock::now();
    opposing_sort<<<num_blocks, threads_per_block>>>(d_data, switch_at, n);
    auto const kernel_end = std::chrono::high_resolution_clock::now();
    CHECK_ERROR("Executing kernel on device");

    // After the timer ends, copy the result back, free the device vector,
    // and echo out the timings and the results.
    cudaMemcpy(data.data(), d_data, sizeof(element_t) * n, cudaMemcpyDeviceToHost);
    CHECK_ERROR("Transferring result back to host");
    cudaFree(d_data);
    CHECK_ERROR("Freeing device memory");

    std::cout << "GPU Solution time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(kernel_end - kernel_start).count()
              << " ns" << std::endl;

    for (auto const x : data) std::cout << x << " ";
    std::cout << std::endl;
}

} // namespace gpu
} // namespace a1
} // namespace csc485b


In [37]:
%%cuda_group_save -g "source" -n "main.cu"
/**
 * Driver for the benchmark comparison. Generates random data,
 * runs the CPU baseline, and then runs your code.
 */

#include <cstddef>  // std::size_t type
#include <iostream> // std::cout, std::endl
#include <vector>

#include "algorithm_choices.h"
#include "data_generator.h"
#include "data_types.h"
#include "cuda_common.h"

int main()
{
    std::size_t const n =32;
    std::size_t const switch_at = 3 * ( n >> 2 ) ;

    auto data = csc485b::a1::generate_uniform< element_t >( n );
    csc485b::a1::cpu::run_cpu_baseline( data, switch_at, n );
    csc485b::a1::gpu::run_gpu_soln( data, switch_at, n );

    return EXIT_SUCCESS;
}

In [38]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

CPU Baseline time: 1120 ns
0 0 1 1 6 7 8 10 10 10 11 13 13 13 15 19 19 23 27 27 31 33 34 35 59 57 56 52 52 51 39 36 
GPU Solution time: 14090 ns
0 0 1 1 6 7 8 10 10 10 11 13 13 13 15 19 19 23 27 27 31 33 34 35 59 57 56 52 52 51 39 36 

